# Model Development Final

In [1]:
import gradio as gr
import matplotlib.pyplot as plt
import pandas as pd 
import pickle
import numpy as np
import warnings
warnings.filterwarnings('ignore')

The application uses two datasets: one for "August" and one for "September."
The "August" dataset includes the actual crime rates.
The "September" dataset includes predicted crime rates.
Both dataset are extracted from the data collection team, while the September dataset is saved from EBM model notebook developed by Xuan Qin where using the predicted crime rate as lag feature to predict September Crime Rate. 

In [2]:
data_dir = "../data/Deployment/" 

In [3]:
august_df = pd.read_csv(data_dir+ "august_predict_data.csv")

In [4]:
sep_df = pd.read_csv(data_dir+ "predicted_september_crime.csv")

Load the EBM model

In [5]:
model_path = "../model/"

In [6]:
def predict_august(data):
    # Predict using the loaded model
    with open(model_path + "ebm.pkl", "rb") as model_file:
        ebm = pickle.load(model_file)
    predictions = ebm.predict(data)
    return predictions

Slice the August dataset so that the EBM model can predict the August Crime Data

In [7]:
X_data2308 = august_df.drop('Offense Count', axis=1)

Function for creating chart

In [8]:
def plot_actual_vs_predicted_august(data,predictions):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    date = data['date'].to_numpy()
    actual = data['Offense Count'].to_numpy()
    
    ax.plot(date, actual, label='Actual', marker='o', linestyle='-', markersize=5)
    ax.plot(date, predictions, label='Predicted', marker='o', linestyle='-', markersize=5)
    ax.set_xticklabels(date, rotation=90)  
    ax.xaxis.set_major_locator(plt.FixedLocator(np.arange(len(date))))
    ax.set_xticklabels(date)
    ax.set_xlabel('Date')
    ax.set_ylabel('Crime Rate')
    ax.set_title('Actual vs. Predicted Crime Rate (August)')
    ax.grid(True)
    ax.legend()
    return fig

In [9]:
def plot_predicted_september(data):
    fig, ax = plt.subplots(figsize=(10, 6))
    
    date = data['date'].to_numpy()
    predicted = data['Offense Count'].to_numpy()
    
    ax.plot(date, predicted, label='Predicted', marker='o', linestyle='-', markersize=5, color='orange')  
    ax.set_xticklabels(date, rotation=90) 
    ax.xaxis.set_major_locator(plt.FixedLocator(np.arange(len(date))))
    ax.set_xticklabels(date)
    ax.set_xlabel('date')
    ax.set_ylabel('Crime Rate')
    ax.set_title('Predicted Crime Rate (September)')
    ax.grid(True)
    ax.legend()
    return fig

Function for creating gradio interface 

In [10]:
def gradio_interface(month):
    if month == 'August':
        predictions = predict_august(X_data2308)
        fig = plot_actual_vs_predicted_august(august_df, predictions)
        table_data = {'Date': X_data2308['date'], 'Predicted': [round(i,0) for i in predictions]}
    else:  # For September
        fig = plot_predicted_september(sep_df)
        table_data = {'Date': sep_df['date'], 'Predicted': sep_df['Offense Count']}
    
    return fig, pd.DataFrame(table_data)

There are 2 tabs, one for home page and the other for EBM visualization. You are free to edit the content of the homepage to make it more meaningful

In [11]:
app1 =  gr.Markdown(
    """
    # Discovering the Relationship Between Crime Statistics and Weather in the Houston Area

    ![Houston Image](file/houston.jpg){width=50}

    *Climate change is a pressing concern for metropolitan areas like Houston, TX, with its rapid population growth posing social, safety, and economic challenges.* The proposed project aims to investigate the relationship between weather and crime statistics in Houston. As global temperatures rise, weather events such as heatwaves, hurricanes, and heavy rainfall may impact crime rates.

    ## Project Background

    ![Data Analysis](file/5031659.jpg)

    *Analyzing historical crime and weather data, the project seeks to provide data-backed insights for law enforcement to address weather-related crime spikes and enhance public safety during extreme weather events.*

    This project's significance lies in its potential to inform crime prevention strategies and resource allocation for law enforcement in Houston.

    **Understanding how weather patterns influence criminal behaviors can lead to targeted approaches in mitigating weather-related crime and building more resilient urban environments.** By leveraging data science and crime analysis, the project contributes to creating safer communities amid the challenges posed by climate change.

    [Learn more about our project](https://omdena.com/chapter-challenges/discover-the-relationship-between-crime-statistics-and-weather-in-the-houston-area/)

    **Join us in this journey to make Houston safer, more resilient, and ready to face the challenges of climate change.**
    """)

In [12]:
app2 = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Dropdown(['August', 'September'], label="Select Month"),
    outputs=[gr.Plot(), gr.Dataframe()],
    live=True,
    title="Crime Rate Prediction and Visualization",
)

In [13]:
demo = gr.TabbedInterface([app1, app2], ["Home Page", "EBM Prediction"])

In [14]:
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://d30d84d768618afa00.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
